In [1]:
import os
import time
import sys
import getpass
from drugrelink import pairs
import pandas as pd

In [2]:
print(sys.version)

3.7.4 (default, Aug 13 2019, 20:35:49) 
[GCC 7.3.0]


In [3]:
print(time.asctime())

Tue Sep 24 17:24:18 2019


In [4]:
print(getpass.getuser())

lxu


In [5]:
from drugrelink import pairs
df = pairs.data_non_overlap(validation_path='/home/lxu/drugrelink/data/validation-statuses.tsv',train_path='/home/lxu/drugrelink/data/transformed-features.tsv.bz2',symptomatic_path='/home/lxu/drugrelink/data/probabilities.tsv')

In [6]:
print(len(df.loc[df['label']==0]),len(df.loc[df['label']==1]))
print(df.head())
print(df['compound'][1])

202313 6855
   Unnamed: 0           compound              disease  label
0           0  Compound::DB01048  Disease::DOID:10652      0
1           1  Compound::DB05812  Disease::DOID:10652      0
2           2  Compound::DB00659  Disease::DOID:10652      0
3           3  Compound::DB00284  Disease::DOID:10652      0
4           4  Compound::DB01193  Disease::DOID:10652      0
Compound::DB05812


In [7]:
nodes_df = pd.read_csv('/home/lxu/drugrelink/data/nodes.tsv',sep='\t')
nodes_list = nodes_df['id'].tolist()
names_list = nodes_df['name'].tolist()
nodes_dict = dict(zip(nodes_list, names_list))

In [8]:
import pickle
from node2vec.edges import HadamardEmbedder
with open('/home/lxu/drugrelink/resources/predictive_model/edge2vec/0/word2vec_model.pickle', 'rb') as file:
    word2vec = pickle.load(file)
def vectors(df):
    vectors=[]
    for _,row in df[['compound','disease']].iterrows():
        c=row['compound']
        
        d=row['disease']
        
   
        edges_embs = HadamardEmbedder(keyed_vectors=word2vec.wv)
        vector = edges_embs[(c,d)]
        vectors.append(vector)
    return vectors

In [9]:
positive = df.loc[df['label']==1]
print(len(positive))
negative = df.loc[df['label'] == 0]
print(len(negative))
new_negative = negative.drop(df.loc[(df['compound']=='Compound::DB02546') & (df['label']== 0)].index)
train_negative = negative.sample(n=len(positive))
hdac6_df= df.loc[(df['compound']=='Compound::DB02546') & (df['label']== 0)]

6855
202313


In [10]:
from sklearn.linear_model import LogisticRegressionCV
import numpy as np

def retrain(X,y):
    model = LogisticRegressionCV().fit(X,y)
    return model


In [11]:
X = vectors(positive) + vectors(train_negative)
y= positive['label'].tolist() + train_negative['label'].tolist()
model = retrain(X,y)
predict_x = vectors(hdac6_df)

/home/lxu/anaconda3/envs/my-new-rdkit-env/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [12]:
import pandas as pd
results = pd.DataFrame({
                      
    'compound': hdac6_df['compound'].tolist(),
    'disease': hdac6_df['disease'].tolist(),
    'label': hdac6_df['label'].tolist(),
    'predict_label': model.predict(predict_x),
    'output_std' : model.predict_proba(predict_x)[:,1]   
}, columns = [ 'compound','disease','label','predict_label','output_std'])

In [13]:
cname= []
dname =[]
for _,row in results[['compound','disease']].iterrows():
    c = nodes_dict[row['compound']]
    d = nodes_dict[row['disease']]
    cname.append(c)
    dname.append(d)
results.insert(loc=1,column='compound_name',value=cname)
results.insert(loc=3,column='disease_name',value=dname)

In [16]:
results

,compound,compound_name,disease,disease_name,label,predict_label,output_std
0,Compound::DB02546,Vorinostat,Disease::DOID:10652,Alzheimer's disease,0,1,0.526692
1,Compound::DB02546,Vorinostat,Disease::DOID:9206,Barrett's esophagus,0,1,0.753817
2,Compound::DB02546,Vorinostat,Disease::DOID:13241,Behcet's disease,0,0,0.228978
3,Compound::DB02546,Vorinostat,Disease::DOID:11949,Creutzfeldt-Jakob disease,0,0,0.368036
4,Compound::DB02546,Vorinostat,Disease::DOID:8778,Crohn's disease,0,1,0.599141
...,...,...,...,...,...,...,...
99,Compound::DB02546,Vorinostat,Disease::DOID:13223,uterine fibroid,0,0,0.471661
100,Compound::DB02546,Vorinostat,Disease::DOID:119,vaginal cancer,0,0,0.223897
101,Compound::DB02546,Vorinostat,Disease::DOID:175,vascular cancer,0,0,0.294554
102,Compound::DB02546,Vorinostat,Disease::DOID:12306,vitiligo,0,0,0.252471


In [17]:
results.loc[(results['label']==0) & (results['predict_label']==1)].sort_values(by='output_std',ascending = False)

,compound,compound_name,disease,disease_name,label,predict_label,output_std
34,Compound::DB02546,Vorinostat,Disease::DOID:219,colon cancer,0,1,0.894500
57,Compound::DB02546,Vorinostat,Disease::DOID:12365,malaria,0,1,0.789760
8,Compound::DB02546,Vorinostat,Disease::DOID:2986,IgA glomerulonephritis,0,1,0.781940
1,Compound::DB02546,Vorinostat,Disease::DOID:9206,Barrett's esophagus,0,1,0.753817
64,Compound::DB02546,Vorinostat,Disease::DOID:4045,muscle cancer,0,1,0.737829
12,Compound::DB02546,Vorinostat,Disease::DOID:7693,abdominal aortic aneurysm,0,1,0.720823
90,Compound::DB02546,Vorinostat,Disease::DOID:418,systemic scleroderma,0,1,0.708142
85,Compound::DB02546,Vorinostat,Disease::DOID:7148,rheumatoid arthritis,0,1,0.695322
45,Compound::DB02546,Vorinostat,Disease::DOID:3121,gallbladder cancer,0,1,0.693198
96,Compound::DB02546,Vorinostat,Disease::DOID:9352,type 2 diabetes mellitus,0,1,0.690350
